<a href="https://colab.research.google.com/github/barakatimothy/Artificial-bee-colony-algorithm-implementation/blob/main/portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance numpy pandas scikit-learn xgboost

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
# Download stock data
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
data = yf.download(tickers, start="2020-01-01", end="2024-01-01")

In [ ]:
simple_return = data.pct_change().dropna()["Close"]
simple_return

In [ ]:
# Correct the order of variables returned by train_test_split
x_train, X_test, y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)
# Train XGBoost Regressor for each stock
models = {}
predicted_returns = {}

for stock in tickers:
    model = XGBRegressor(objective="reg:squarederror", n_estimators=100)
    # Now using the correct training data and labels
    model.fit(x_train, y_train[stock])
    models[stock] = model
    # Predict on the correct test data
    predicted_returns[stock] = model.predict(X_test)  # Get predicted return

In [ ]:
class ABCPortfolioOptimizer:
    def __init__(self, assets, predicted_returns, pop_size=30, max_iter=100):
        self.assets = assets
        self.predicted_returns = predicted_returns
        self.pop_size = pop_size
        self.max_iter = max_iter
        self.portfolios = np.random.dirichlet(np.ones(len(assets)), size=pop_size)
        # Calculate and store returns during initialization
        self.returns = simple_return[self.assets]

    def fitness(self, weights):
        """Objective function: Maximize Sharpe Ratio"""
        portfolio_return = np.dot(weights, self.predicted_returns)
        # Use self.returns for risk calculation
        portfolio_risk = np.sqrt(np.dot(weights.T, np.cov(self.returns.T) @ weights))
        sharpe_ratio = portfolio_return / portfolio_risk if portfolio_risk > 0 else 0
        # Ensure sharpe_ratio is non-negative
        return max(0, sharpe_ratio)  # Clip negative values to 0

    def employ_bees(self):
        """Employed bees improve their solutions."""
        for i in range(self.pop_size):
            new_portfolio = self.portfolios[i] + np.random.uniform(-0.1, 0.1, len(self.assets))
            new_portfolio = np.clip(new_portfolio, 0, 1)  # Keep weights valid
            new_portfolio /= new_portfolio.sum()  # Normalize
            if self.fitness(new_portfolio) > self.fitness(self.portfolios[i]):
                self.portfolios[i] = new_portfolio

    def onlooker_bees(self):
        """Onlooker bees exploit good solutions."""
        fitness_values = np.array([self.fitness(p) for p in self.portfolios])
        # Replace zeros with small positive values to avoid division by zero
        fitness_values = np.where(fitness_values == 0, 1e-10, fitness_values)
        probabilities = fitness_values / fitness_values.sum()
        for i in range(self.pop_size):
            selected = np.random.choice(self.pop_size, p=probabilities)
            new_portfolio = self.portfolios[selected] + np.random.uniform(-0.05, 0.05, len(self.assets))
            new_portfolio = np.clip(new_portfolio, 0, 1)
            new_portfolio /= new_portfolio.sum()
            if self.fitness(new_portfolio) > self.fitness(self.portfolios[selected]):
                self.portfolios[selected] = new_portfolio

    def scout_bees(self):
        """Scout bees introduce diversity by replacing bad solutions."""
        worst_index = np.argmin([self.fitness(p) for p in self.portfolios])
        self.portfolios[worst_index] = np.random.dirichlet(np.ones(len(self.assets)))

    def optimize(self):
        """Run the ABC algorithm."""
        best_portfolio = None
        best_fitness = -np.inf

        for _ in range(self.max_iter):
            self.employ_bees()
            self.onlooker_bees()
            self.scout_bees()

            # Track the best solution
            current_best = max(self.portfolios, key=self.fitness)
            current_fitness = self.fitness(current_best)
            if current_fitness > best_fitness:
                best_fitness = current_fitness
                best_portfolio = current_best

        return best_portfolio, best_fitness

In [ ]:
# Get average predicted returns
mean_predicted_returns = np.array([np.mean(predicted_returns[stock]) for stock in tickers])

# Run ABC optimization
abc_optimizer = ABCPortfolioOptimizer(tickers, mean_predicted_returns, pop_size=50, max_iter=200)
best_portfolio, best_sharpe = abc_optimizer.optimize()

# Display results
print("Optimal Portfolio Weights:")
for stock, weight in zip(tickers, best_portfolio):
    print(f"{stock}: {weight:.4f}")

print(f"Maximized Sharpe Ratio: {best_sharpe:.4f}")
